In [ ]:
import openmm
from ipywidgets import Image
from openff.interchange.components._packmol import (
    RHOMBIC_DODECAHEDRON,
    solvate_topology,
)
from openff.toolkit import ForceField, Molecule, Topology
from openff.units import Quantity
from rdkit.Chem import Draw
from rdkit.Chem.rdChemReactions import ReactionFromSmarts

In [ ]:
from openff.pablo import CCD_RESIDUE_DEFINITION_CACHE
from ptm_prototype import draw_molecule

cysteine = CCD_RESIDUE_DEFINITION_CACHE["CYS"][0].to_openff_molecule()
draw_molecule(
    cysteine,
    width=700,
    height=600,
    atom_notes={
        i: (
            f"{i}:{atom.name}"
            + f"{'^' if bool(atom.metadata.get('leaving_atom')) else ''}"
        )
        for i, atom in enumerate(cysteine.atoms)
    },
)

In [ ]:
maleimide = Molecule.from_file("maleimide.sdf")
maleimide.generate_unique_atom_names()

draw_molecule(
    maleimide,
    width=700,
    height=600,
    atom_notes={
        i: (
            f"{i}:{atom.name}"
            + f"{'^' if bool(atom.metadata.get('leaving_atom')) else ''}"
        )
        for i, atom in enumerate(maleimide.atoms)
    },
)

In [ ]:
thiol_maleimide_click_smarts = (
    "[C:10]-[S:1]-[H:2]"
    + "."
    + "[N:3]1-[C:4](=[O:5])-[C:6](-[H:11])=[C:7](-[H:12])-[C:8](=[O:9])-1"
    + ">>"
    + "[N:3]1-[C:4](=[O:5])-[C:6](-[H:2])(-[H:11])-[C@:7](-[S:1]-[C:10])(-[H:12])-[C:8](=[O:9])-1"
)

rxn = ReactionFromSmarts(thiol_maleimide_click_smarts)
d2d = Draw.MolDraw2DCairo(800, 300)
d2d.DrawReaction(
    ReactionFromSmarts(thiol_maleimide_click_smarts), highlightByReactant=True
)
Image(value=d2d.GetDrawingText())

In [ ]:
from ptm_prototype import react

products = list(react([cysteine, maleimide], thiol_maleimide_click_smarts))
dye = products[0][0]


draw_molecule(
    dye,
    width=700,
    height=600,
    atom_notes={
        i: (
            f"{i}:{atom.name}"
            + f"{'^' if bool(atom.metadata.get('leaving_atom')) else ''}"
        )
        for i, atom in enumerate(dye.atoms)
    },
)

Atom names are tricky; we've generated atom names automatically, but they almost certainly won't match what you have in a PDB. You could:

1. Modify the PDB to use the above atom names, or
2. Modify the substructure atom names to match the PDB file, or
3. Match based on connectivity rather than atom names. This requires CONECT records in the PDB, and is not yet supported by the new loader (coming soon!)

In [ ]:
name_corrections = {
    4: "H3x",
    25: "C9x",
    26: "H4x",
    28: "C8x",
    29: "H6x",
    30: "H5x",
    32: "C10x",
    33: "C11x",
    34: "O2x",
    35: "O3x",
    36: "C23x",
    37: "C12x",
    38: "C22x",
    39: "C18x",
    40: "C13x",
    41: "C17x",
    42: "H14x",
    43: "C21x",
    44: "C19x",
    45: "O5x",
    46: "C14x",
    47: "H7x",
    49: "H13x",
    50: "C20x",
    51: "H11x",
    52: "C15x",
    53: "H8x",
    54: "H10x",
    55: "O6x",
}

for i, name in name_corrections.items():
    # assert dye.atom(i).name != name, f"{i}:{name}=={dye.atom(i).name}"
    dye.atom(i).name = name

draw_molecule(
    dye,
    width=700,
    height=600,
    atom_notes={
        i: (
            f"{i}:{atom.name}"
            + f"{'^' if bool(atom.metadata.get('leaving_atom')) else ''}"
        )
        for i, atom in enumerate(dye.atoms)
    },
)

In [ ]:
from openff.pablo import ResidueDefinition
from openff.pablo.chem import PEPTIDE_BOND

dye_resdef = ResidueDefinition.from_molecule(
    molecule=dye,
    residue_name="DYE",
    linking_bond=PEPTIDE_BOND,
)

In [ ]:
from openff.pablo import CCD_RESIDUE_DEFINITION_CACHE, topology_from_pdb

topology = topology_from_pdb(
    "3ip9_dye.pdb",
    residue_database=CCD_RESIDUE_DEFINITION_CACHE.with_({"DYE": [dye_resdef]}),
)

In [ ]:
w = topology.visualize()
w.clear_representations()
w.add_cartoon()
w.add_line(opacity=0.5, crossSize=1.0)
w.add_licorice("DYE", radius=0.3)
w.add_unitcell()
w.center("DYE")
w

This is much faster than the existing implementation - and the difference is even more dramatic with solvated PDB files:

In [ ]:
substructure_mol = dye_resdef.to_openff_molecule()

In [ ]:
legacy_topology = Topology.from_pdb(
    "3ip9_dye.pdb",
    _additional_substructures=[substructure_mol],
)

In [ ]:
w = legacy_topology.visualize()
w.clear_representations()
w.add_cartoon()
w.add_line(opacity=0.5, crossSize=1.0)
w.add_licorice("DYE", radius=0.3)
w.add_unitcell()
w.center("DYE")
w

## Solvation

In [ ]:
topology = solvate_topology(
    topology,
    nacl_conc=Quantity(0.1, "mol/L"),
    padding=Quantity(1.2, "nm"),
    box_shape=RHOMBIC_DODECAHEDRON,
)

In [ ]:
w = topology.visualize()
w.clear_representations()
w.add_cartoon()
w.add_line(opacity=0.5, crossSize=1.0)
w.add_licorice("DYE", radius=0.3)
w.add_unitcell()
w.center("DYE")
w

## Assigning parameters

In [ ]:
from ptm_prototype import parametrize_with_nagl

sage_ff14sb = ForceField("openff-2.2.1.offxml", "ff14sb_off_impropers_0.0.4.offxml")

interchange = parametrize_with_nagl(force_field=sage_ff14sb, topology=topology)

In [ ]:
from ptm_prototype import get_openmm_total_charge

temperature = 300 * openmm.unit.kelvin
pressure = 1 * openmm.unit.bar

timestep = 2 * openmm.unit.femtosecond
friction_coeff = 1 / openmm.unit.picosecond
barostat_frequency = 25

print("making OpenMM simulation ...")
simulation = interchange.to_openmm_simulation(
    integrator=openmm.LangevinMiddleIntegrator(
        temperature,
        friction_coeff,
        timestep,
    ),
    additional_forces=[
        openmm.MonteCarloBarostat(
            pressure,
            temperature,
            barostat_frequency,
        ),
    ],
)

print(f"total system charge is {get_openmm_total_charge(simulation.system)}")

print("serializing OpenMM system ...")
with open("system.xml", "w") as f:
    f.write(openmm.XmlSerializer.serialize(simulation.system))

In [ ]:
dcd_reporter = openmm.app.DCDReporter("trajectory.dcd", 100)
simulation.reporters.append(dcd_reporter)

simulation.context.computeVirtualSites()
simulation.minimizeEnergy()
simulation.context.setVelocitiesToTemperature(simulation.integrator.getTemperature())

In [ ]:
simulation.runForClockTime(1.0 * openmm.unit.minute)

In [ ]:
from ptm_prototype import nglview_show_openmm

w = nglview_show_openmm(simulation.topology, "trajectory.dcd")
w.add_licorice("DYE", radius=0.3)
w.center("DYE")
w

In [ ]:
# TODO: Test in vanilla `openff-toolkit-examples` environment